## Preparation

In [4]:
!pip install mediapipe==0.10.15

In [6]:
# Imports
import cv2
import time
import mediapipe as mp



In [15]:
print(dir(mp.tasks))

['BaseOptions', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'audio', 'components', 'genai', 'text', 'vision']


In [17]:
#from mediapipe.tasks import vision
from mediapipe.tasks import BaseOptions
from mediapipe.tasks.vision.object_detector import ObjectDetector
from mediapipe.tasks.vision.object_detector import ObjectDetectorOptions

from mediapipe.tasks.components.processors import running_mode

ImportError: cannot import name 'BaseOptions' from 'mediapipe.tasks' (/opt/anaconda3/lib/python3.12/site-packages/mediapipe/tasks/__init__.py)


CondaError: Run 'conda init' before 'conda activate'


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from utils import streaming_utils_Tut
streaming_utils_Tut.getStream(0)

cap = streaming_utils_Tut.getStream()

In [ ]:



# Access enum like:
mode = running_mode.RunningMode.LIVE_STREAM


# Callback function to handle detection results
def detection_callback(result, output_image, timestamp_ms):
    print("🔎 Detection at", timestamp_ms, "ms")
    for det in result.detections:
        category = det.categories[0]
        print(f" - {category.category_name}: {category.score:.2f}")

# Load model
MODEL_PATH = '/Users/intern5/Downloads/efficientdet_lite0-2.tflite'

# Configure options
options = ObjectDetectorOptions(
    base_options=BaseOptions(model_asset_path=MODEL_PATH),
    running_mode=RunningMode.LIVE_STREAM,
    score_threshold=0.5,
    max_results=3,
    result_callback=detection_callback
)

# Create detector
detector = ObjectDetector.create_from_options(options)

# Start webcam

timestamp = 0

try:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        timestamp = int(time.time() * 1000)
        # Convert image to MediaPipe format
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)

        # Async detect
        detector.detect_async(mp_image, timestamp)

        # Show image
        cv2.imshow("Live Object Detection", frame)
        if cv2.waitKey(5) & 0xFF == 27:  # Press Esc to exit
            break
finally:
    cap.release()
    cv2.destroyAllWindows()
    detector.close()


In [ ]:
import cv2
import mediapipe as mp
import os
# print("before")

model_path = '/Users/intern5/Downloads/efficientdet_lite0-2.tflite'
print("before")
# Set up MediaPipe components
BaseOptions = mp.tasks.BaseOptions
DetectionResult = mp.tasks.vision.ObjectDetectorResult
ObjectDetector = mp.tasks.vision.ObjectDetector
ObjectDetectorOptions = mp.tasks.vision.ObjectDetectorOptions
VisionRunningMode = mp.tasks.vision.RunningMode.LIVE_STREAM
Image = mp.Image

# Callback function for results
def print_result(result: DetectionResult, output_image: mp.Image, timestamp_ms: int):
    print('Detection result: {}', result)

print("before1")

# Define the model path
# model_path = 'efficientdet_lite0.tflite'
# Check if the model file exists
if not os.path.exists(model_path):
    print(f"Error: Model file not found at {model_path}. Please ensure the wget command downloaded it successfully.")
    # You might want to exit or handle this error more gracefully
    exit() # Exit if the file is not found

# ✅ Configure detector options
try:
    print("[INFO] Creating ObjectDetector options...")
    # Setup options
    options = ObjectDetectorOptions(
        base_options=BaseOptions(model_asset_path=model_path),
        running_mode=VisionRunningMode.LIVE_STREAM,
        max_results=5,
        result_callback=print_result
    )
except Exception as e:
    print("[ERROR] Failed to create ObjectDetectorOptions.")
    traceback.print_exc()
    exit()

# OpenCV video capture
# cap = cv2.VideoCapture(0, cv2.CAP_QT)  # Change to a video file path if needed
# if not cap.read()[0]:
#     print("[ERROR] Cannot open webcam.")

# Initialize detector
with ObjectDetector.create_from_options(options) as detector:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert BGR to RGB (MediaPipe expects RGB)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Create a MediaPipe Image object
        mp_image = Image(image_format=Image.ImageFormat.SRGB, data=frame_rgb)

        # Get timestamp in milliseconds
        timestamp_ms = int(cap.get(cv2.CAP_PROP_POS_MSEC))

        # Run async detection
        detector.detect_async(mp_image, timestamp_ms)

        # Optionally, display the frame
        cv2.imshow('Webcam Feed', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
